## <center>This Model will Attempt to Predict the Type of Bicycle Used from Trip Data</center>

In [1]:
# Eliminating Warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Importing Libraries
import pandas as pd
import numpy as np
import os
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from imblearn.over_sampling import SMOTE
from sklearn.metrics import balanced_accuracy_score

In [3]:
# Set up data path
data_path = 'data'

In [4]:
# Load base data to dataframe
all_data_file = 'all_data.csv'
all_df = pd.read_csv(os.path.join(data_path, all_data_file))
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
1,10/21/16,Citibike from Work,3.51,NaN,26.0,0.43,8.10,Citibike
2,5/17/21,Dn ES Up LWS,16.27,1.0,57.0,1.95,8.34,Specialized
3,7/6/19,Down ES Up WS,16.12,2.0,12.0,2.20,7.33,Specialized
4,8/25/16,Citibike to Work,1.18,NaN,10.0,0.17,7.08,Citibike
...,...,...,...,...,...,...,...,...
358,7/13/16,Citibike to Work,1.45,NaN,10.0,0.17,8.70,Citibike
359,6/7/19,CP BL,7.47,NaN,54.0,0.90,8.30,Specialized
360,4/26/21,Dn UWS Up ES,20.85,2.0,34.0,2.57,8.12,Specialized
361,6/6/20,CP BL,6.14,NaN,37.0,0.62,9.96,Specialized


In [5]:
# Change NaN values to 0 - only occurs on hours column so this is accurate
all_df['Hours'] = all_df['Hours'].fillna(0)
all_df

,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
1,10/21/16,Citibike from Work,3.51,0.0,26.0,0.43,8.10,Citibike
2,5/17/21,Dn ES Up LWS,16.27,1.0,57.0,1.95,8.34,Specialized
3,7/6/19,Down ES Up WS,16.12,2.0,12.0,2.20,7.33,Specialized
4,8/25/16,Citibike to Work,1.18,0.0,10.0,0.17,7.08,Citibike
...,...,...,...,...,...,...,...,...
358,7/13/16,Citibike to Work,1.45,0.0,10.0,0.17,8.70,Citibike
359,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
360,4/26/21,Dn UWS Up ES,20.85,2.0,34.0,2.57,8.12,Specialized
361,6/6/20,CP BL,6.14,0.0,37.0,0.62,9.96,Specialized


In [6]:
# Turning Index into Orig_Index Column
all_df.reset_index(inplace=True)
all_df = all_df.rename(columns = {'index': 'Orig_Index'})
all_df

,Orig_Index,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type
0,0,7/17/16,Down ES UP WS,17.67,2.0,5.0,2.08,8.48,Specialized
1,1,10/21/16,Citibike from Work,3.51,0.0,26.0,0.43,8.10,Citibike
2,2,5/17/21,Dn ES Up LWS,16.27,1.0,57.0,1.95,8.34,Specialized
3,3,7/6/19,Down ES Up WS,16.12,2.0,12.0,2.20,7.33,Specialized
4,4,8/25/16,Citibike to Work,1.18,0.0,10.0,0.17,7.08,Citibike
...,...,...,...,...,...,...,...,...,...
358,358,7/13/16,Citibike to Work,1.45,0.0,10.0,0.17,8.70,Citibike
359,359,6/7/19,CP BL,7.47,0.0,54.0,0.90,8.30,Specialized
360,360,4/26/21,Dn UWS Up ES,20.85,2.0,34.0,2.57,8.12,Specialized
361,361,6/6/20,CP BL,6.14,0.0,37.0,0.62,9.96,Specialized


In [7]:
# Defining fields to use
columns = ['Miles', 'Duration', 'Speed','Type']
# target = ['Type']

In [8]:
# Create Dataframe with desired columns
bike_df = all_df.loc[:,columns].copy()
bike_df

,Miles,Duration,Speed,Type
0,17.67,2.08,8.48,Specialized
1,3.51,0.43,8.10,Citibike
2,16.27,1.95,8.34,Specialized
3,16.12,2.20,7.33,Specialized
4,1.18,0.17,7.08,Citibike
...,...,...,...,...
358,1.45,0.17,8.70,Citibike
359,7.47,0.90,8.30,Specialized
360,20.85,2.57,8.12,Specialized
361,6.14,0.62,9.96,Specialized


In [9]:
# Create Dataframe with desired columns
# Demonstrating different way to do it
# bike_df = all_df[['Miles', 'Duration', 'Speed','Type']]
# bike_df

In [9]:
# Splitting out Features and Target
y = bike_df["Type"]
X = bike_df.drop(columns="Type")

In [10]:
# Splitting out Training and Testing data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

In [11]:
# Checking training data shape
X_train.shape

(272, 3)

In [12]:
# Checking testing data shape
X_test.shape

(91, 3)

In [13]:
# Creating the Logistic Regression Model
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(solver='lbfgs', max_iter=200, random_state=1)

In [14]:
# Training the Model
classifier.fit(X_train, y_train)

LogisticRegression(max_iter=200, random_state=1)

In [15]:
# Create Predictions
y_pred = classifier.predict(X_test)

In [16]:
# Get accuracy score
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred))

0.978021978021978


In [17]:
# Convert Predictions to DataFrame
pred_df = pd.DataFrame(y_pred)
pred_df.reset_index()
pred_df = pred_df.rename(columns = {0:'Prediction'})
pred_df

,Prediction
0,Citibike
1,Specialized
2,Citibike
3,Citibike
4,Citibike
...,...
86,Specialized
87,Specialized
88,Specialized
89,Specialized


In [18]:
# Convert Tests to DataFrame
test_df = pd.DataFrame(y_test)
test_df

,Type
22,Citibike
205,Specialized
332,Citibike
136,Citibike
324,Citibike
...,...
88,Specialized
325,Specialized
135,Specialized
84,Specialized


In [19]:
# Making index into a column
test_df.reset_index(inplace=True)
test_df = test_df.rename(columns = {'index': 'Orig_Index'})
test_df

,Orig_Index,Type
0,22,Citibike
1,205,Specialized
2,332,Citibike
3,136,Citibike
4,324,Citibike
...,...,...
86,88,Specialized
87,325,Specialized
88,135,Specialized
89,84,Specialized


In [20]:
# Combining the new test and predition dataframes horizontally for comparison
test_pred_df = pd.concat([test_df, pred_df], axis = 1)
test_pred_df

,Orig_Index,Type,Prediction
0,22,Citibike,Citibike
1,205,Specialized,Specialized
2,332,Citibike,Citibike
3,136,Citibike,Citibike
4,324,Citibike,Citibike
...,...,...,...
86,88,Specialized,Specialized
87,325,Specialized,Specialized
88,135,Specialized,Specialized
89,84,Specialized,Specialized


In [21]:
# Identify records where prediction is wrong
pred_errs_df = test_pred_df.loc[test_pred_df['Type'] != test_pred_df['Prediction']]
pred_errs_df

,Orig_Index,Type,Prediction
19,117,Specialized,Citibike
53,264,Specialized,Citibike


In [22]:
# Merging pred_errs_df with all_df to see details of errors
err_details_df = pd.merge(pred_errs_df, all_df, on=["Orig_Index", "Orig_Index"])
err_details_df

,Orig_Index,Type_x,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed,Type_y
0,117,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80,Specialized
1,264,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54,Specialized


In [23]:
# Cleaning up err_details_df to eliminate redundancy and improve readability
err_details_df.drop(columns='Type_y', inplace=True)
err_details_df.rename(columns = {'Type_x' : 'Type'}, inplace=True)
err_details_df

,Orig_Index,Type,Prediction,Date,Destination,Miles,Hours,Minutes,Duration,Speed
0,117,Specialized,Citibike,7/15/19,Up Down ES,3.74,0.0,33.0,0.55,6.80
1,264,Specialized,Citibike,7/28/19,Roosevelt Island,4.25,0.0,39.0,0.65,6.54
